---
># AFP_2022_2_K_10_Akropolisz Gépi tanulás framework
---
Hadobás Dávid (TB3376), Kardos Zsolt (O48WRX), Riczkó Henrik (D5GPJ6), Balogh Mihály Viktor (GUFVXA)
---

In [2]:
import math, numpy, random
adatbazis_helye = "xd" #@param {type:"string"}
#@markdown --- 
algoritmus = 'A' #@param ["A", "B", "C"]
aktivaciosfgv = 'sigmoid' #@param ['tanh', 'sigmoid']
Bs = 0 #@param {type:"number"}
nnszelesseg = 4 #@param {type:"number"}
nnmagassag = 3 #@param {type:"number"}
sulymin = 5 #@param {type:"number"}
sulymax = 7 #@param {type:"number"}
print('Választott algoritmus: ', algoritmus)
print('Választott aktivációs függvény: ', aktivaciosfgv)
print('Választott BIAS: ', Bs)
print('Választott szélesség: ', nnszelesseg)

Választott algoritmus:  A
Választott aktivációs függvény:  sigmoid


## Jelen állapotban az első kódblokkot a drop-down menük opciói kiválasztása után újra kell futtatni!!!!

In [ ]:
if aktivaciosfgv == 'tanh':
  def activation(x):
    return numpy.tanh(x)
  def dactivation(x):
    return 1.0 - x**2
elif aktivaciosfgv == 'sigmoid':
  def activation(x):
    return 1.0/(1.0 + numpy.exp(-x))
  def dactivation(x):
    return x*(1.0-x)

'''Bővíthetőség miatt maradnak elif-ek'''
if algoritmus == 'A':
  acti, dacti = activation, dactivation

  ''' Zsolti: Ide jöhet majd az adatbázis olvasás '''
  #samples = [[[0, 0], [0, 0]], [[0, 1], [0, 1]], [[1, 0], [0, 1]], [[1, 1], [1, 1]]]   # AND / OR
  samples = [ [[0, 0], [0, 0, 0]], [[0, 1], [0, 1, 1]], [[1, 0], [0, 1, 1]], [[1, 1], [1, 1, 0]] ]  # AND / OR / XOR
  #samples = numpy.loadtxt(fname="db.csv", delimiter=",") //a CSV olvasáshoz

  B = Bs
  nn = [len(samples[0][0])+B, nnszelesseg, nnmagassag, len(samples[0][1])]
  if sulymax<sulymin:
    print('hiba')
  wl = [ numpy.random.random((nn[l+1], nn[l]))*sulymax-sulymin for l in range(len(nn)-1)] 
  delta = [numpy.zeros((nn[l+1])) for l in range(len(nn)-1)]

  epoch = 0
  sumerr = 1.0
  while sumerr>=0.01 and epoch<=10000:
    sumerr = 0.0
    epoch += 1
    for inp, out in samples:
        nl = [ numpy.array(inp + [1.0]*B) ]
        for l in range(len(nn)-1):
            nl.append(acti(numpy.dot(wl[l],nl[l])))
        error = out - nl[-1]
        for l in reversed(range(len(nn)-1)):
            if l == len(nn)-2:
                delta[l][:] = error*dacti(nl[-1])
            else:
                numpy.dot(delta[l+1],wl[l+1], out=delta[l])
                delta[l] *= dacti(nl[l+1])
            wl[l] += 0.5 * delta[l].reshape((nn[l+1],1))*nl[l].reshape((1,nn[l]))
        sumerr += sum(error**2)
  print (epoch,sumerr)
elif algoritmus == 'B':

  acti, dacti = activation, dactivation
  #Adatbázist ide - Henrik
  #samples = [[[0, 0], [0, 0]], [[0, 1], [0, 1]], [[1, 0], [0, 1]], [[1, 1], [1, 1]]]   # AND és OR
  samples = [ [[0, 0], [0, 0, 0]], [[0, 1], [0, 1, 1]], [[1, 0], [0, 1, 1]], [[1, 1], [1, 1, 0]] ]  # AND, OR és XOR
  #samples = numpy.loadtxt(fname="db.csv", delimiter=",") //a CSV olvasáshoz

  B = Bs
  nn = [len(samples[0][0])+B, nnszelesseg, nnmagassag, len(samples[0][1])]
  wl=[ [ [random.random()*sulymax-sulymin for _ in range(nn[l])] for _ in range(nn[l+1])] for l in range(len(nn)-1)] 
  epoch = 0
  sumerr = 1.0
  while sumerr>=0.01 and epoch<=10000:
      sumerr = 0.0
      epoch += 1
      for inp, out in samples:
          nl = [ inp + [1.0]*B ]
          for l in range(len(nn)-1):
              nl.append([acti(sum([nl[l][i] * wl[l][j][i] for i in range(nn[l])])) for j in range(nn[l+1])])
              
          error = [out[j] - nl[-1][j] for j in range(nn[-1])]
          delta = [None for _ in range(len(nn)-1)]
          for l in reversed(range(len(nn)-1)):
              if l == len(nn)-2:
                  delta[l] = [error[j] * dacti(nl[-1][j]) for j in range(nn[-1])]
              else:
                  delta[l] = [sum([delta[l+1][j] * wl[l+1][j][i] for j in range(nn[l+2])])*dacti(nl[l+1][i]) for i in range(nn[l+1])]
              
              for i in range(nn[l]):
                  for j in range(nn[l+1]):
                      wl[l][j][i] += 0.5 * delta[l][j] * nl[l][i]

          sumerr += sum( [error[j]**2 for j in range(nn[-1])])
  print (epoch,sumerr)

elif algoritmus == 'C':
  acti, dacti = activation, dactivation

  w1 = [random.random()*sulymax-sulymin for _ in range(3)]
  #Adatbázist ide - Henrik
  #samples = [[[0.0, 0.0], 0.0], [[0.0, 1.0], 1.0], [[1.0, 0.0], 1.0], [[1.0, 1.0], 1.0]] OR
  samples = [[[0.0, 0.0], 0.0], [[0.0, 1.0], 0.0], [[1.0, 0.0], 0.0], [[1.0, 1.0], 1.0]] #AND
  #samples = [[[0.0, 0.0], 0.0], [[0.0, 1.0], 1.0], [[1.0, 0.0], 1.0], [[1.0, 1.0], 0.0]] XOR
  #samples = numpy.loadtxt(fname="db.csv", delimiter=",") //a CSV olvasáshoz

  for _ in range(100):
      sumerr = 0.0
      for inp, out in samples:
          x = inp + [1.0]
          y = acti(sum([x[i] * w1[i] for i in range(3)]))
          error = (out - y)
          delta = error * dacti(y)
          for i in range(3):
              w1[i] += 0.8 * delta * x[i]
          sumerr += (y-out)**2
      print (sumerr)

hiba
10001 3.0
